# Lesson 3: Can a large language model master Wordle?

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code>  file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

Start by load dependencies and setting up the Predibase client, which you'll use to call both base and finetuned models:

In [4]:
import os

from dotenv import load_dotenv
from openai import OpenAI

# Initialize client to query Qwen2.5 7B Instruct on Predibase using the OpenAI client.

_ = load_dotenv()

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"], # OpenAI API key
)

Load the model tokenizer from Hugging face:

In [7]:
import tiktoken

# Load the GPT-4o-mini tokenizer
# Note: GPT-4o-mini uses the cl100k_base encoding
tokenizer = tiktoken.get_encoding("cl100k_base")

# Example usage
text = "Hello, this is a test for GPT-4o-mini tokenizer."
tokens = tokenizer.encode(text)

# Print results
print(f"Text: {text}")
print(f"Token IDs: {tokens}")
print(f"Token count: {len(tokens)}")

# Decode back to text
decoded_text = tokenizer.decode(tokens)
print(f"Decoded text: {decoded_text}")

Text: Hello, this is a test for GPT-4o-mini tokenizer.
Token IDs: [9906, 11, 420, 374, 264, 1296, 369, 480, 2898, 12, 19, 78, 68454, 47058, 13]
Token count: 15
Decoded text: Hello, this is a test for GPT-4o-mini tokenizer.


## Setting up prompts to play Wordle

In [8]:
SYSTEM_PROMPT = """
You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close 
your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to 
first add your step by step thought process within <think> </think> 
tags. Then, return your guessed word in the following format: 
<guess> guessed-word </guess>.
"""


In [9]:
from dataclasses import dataclass
from enum import Enum
from typing import List


class LetterFeedback(Enum):
    CORRECT = "✓"
    WRONG_POS = "-"
    WRONG_LETTER = "x"


@dataclass
class GuessWithFeedback:
    guess: str
    feedback: List[LetterFeedback]

    def __repr__(self) -> str:
        """Returns a readable string showing the guess alongside
        its letter-by-letter feedback."""
        feedback_str = " ".join(f"{letter}({fb.value})" for letter, fb in zip(self.guess, self.feedback))
        return f"{self.guess} → Feedback: {feedback_str}"

In [10]:
def render_user_prompt(past_guesses: List[GuessWithFeedback]) -> str:
    """Creates a user-facing prompt that includes past guesses 
    and their feedback."""
    prompt = "Make a new 5-letter word guess."
    if past_guesses:
        prompt += "\n\nHere is some previous feedback:"
        for i, guess in enumerate(past_guesses):
            prompt += f"\nGuess {i+1}: {guess}"
    return prompt

In [14]:
def render_prompt(past_guesses: List[GuessWithFeedback]):
    """Formats a full chat prompt using a system message, user 
    prompt, and assistant preamble to start the model's 
    step-by-step reasoning."""
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": render_user_prompt(past_guesses)
        },
        {
            "role": "assistant",
            "content": "Let me solve this step by step.\n<think>"
        }
    ]

    return messages

In [15]:
def generate_stream(prompt: str, adapter_id: str = "") -> str:
    """Streams a model-generated response from a prompt in 
    real-time and prints it as it arrives."""
    response = client.completions.create(
        model=adapter_id,
        prompt=prompt,
        # Produce deterministic responses for evaluation
        temperature=0.0, 
        max_tokens=2048,
        stream=True,
    )
    
    completion = ""
    for chunk in response:
        if chunk.choices[0].text is not None:
            content = chunk.choices[0].text
            print(content, end="", flush=True)
            completion += content
    print()

    return completion

## Play a single turn of Wordle

Start by setting up the prompt with feedback for two prior guesses:

In [17]:
secret_word = "CRAFT"

past_guesses = [
    GuessWithFeedback(
        "CRANE", [
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.WRONG_LETTER, 
            LetterFeedback.WRONG_LETTER,
        ]),
    GuessWithFeedback(
        "CRASH", [
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.CORRECT, 
            LetterFeedback.WRONG_LETTER, 
            LetterFeedback.WRONG_LETTER,
        ]),
]

prompt = render_prompt(past_guesses)
print(prompt)

[{'role': 'system', 'content': '\nYou are playing Wordle, a word-guessing game.\n\n### Game Rules:\n- You have **6 tries** to guess a secret **5-letter** word.\n- Each guess must be a valid **5-letter English word**.\n- After each guess, you will receive feedback indicating how close \nyour guess was.\n\n### Feedback Format:\nEach letter in your guess will receive one of three symbols:\n1. ✓ : The letter is in the word and in the CORRECT position.\n2. - : The letter is in the word but in the WRONG position.\n3. x : The letter is NOT in the word.\n\n### Example:\nSecret Word: BRISK\n\nGuess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)\nGuess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)\nGuess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)\n\n### Response Format:\nThink through the problem and feedback step by step. Make sure to \nfirst add your step by step thought process within <think> </think> \ntags. Then, return your guessed word in the following format: \n<guess> guessed-word </g

Prompt the base model and examine its response:

In [18]:
base_completion = generate_stream(prompt)

BadRequestError: Error code: 400 - {'error': {'message': 'you must provide a model parameter', 'type': 'invalid_request_error', 'param': None, 'code': None}}

Now prompt the RFT model and see how it responds:

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> <b>NOTE:</b> The cell below loads a model that has been tuned using RFT to play wordle. You'll learn how this model was trained throughout the course, and see the exact setup used to train the model on the Predibase platform in L8.
</div>

In [10]:
# prompt fine-tuned model
ft_completion = generate_stream(prompt, adapter_id="wordle-dlai/2")

Let's analyze the feedback we have so far:

Guess 1: CRANE → Feedback: C(✓) R(✓) A(✓) N(x) E(x)
This means:
- C is in the correct position (1st position)
- R is in the correct position (2nd position)
- A is in the correct position (3rd position)
- N is not in the word
- E is not in the word

Guess 2: CRASH → Feedback: C(✓) R(✓) A(✓) S(x) H(x)
This confirms:
- C, R, A are in their correct positions (1st, 2nd, 3rd)
- S is not in the word
- H is not in the word

So we know:
- The first three letters are "CRA"
- The letters N, E, S, H are not in the word

Now I need to make a guess for the 4th and 5th positions. Let me think of 5-letter words that start with "CRA" and don't contain N, E, S, H.

Possible words:
- CRAFT
- CRAMP
- CRANK (no, contains N)
- CRAP
- CRAZE (no, contains E)
- CRAZY (no, contains Z)
- CRAB
- CRACK
- CRAM
- CRAYON (no, contains N)

Let's eliminate those with letters we know are not in the word.

From the remaining options, CRAFT, CRAMP, CRAP, CRAB, CRACK, and CRAM se

## Playing a game of wordle

Start by setting up a helper function that gets feedback on a guess:

In [11]:
import re

def get_feedback(guess: str, secret_word: str) -> List[LetterFeedback]:
    valid_letters = set(secret_word)
    feedback = []
    for letter, secret_letter in zip(guess, secret_word):
        if letter == secret_letter:
            feedback.append(LetterFeedback.CORRECT)
        elif letter in valid_letters:
            feedback.append(LetterFeedback.WRONG_POS)
        else:
            feedback.append(LetterFeedback.WRONG_LETTER)
    return feedback

Now create a `next_turn` function that incorporates feedback on a guess into the prompt to the LLM:

In [12]:
def next_turn(
    past_guesses: List[GuessWithFeedback], 
    secret_word: str, 
    adapter_id = ""
):
    prompt = render_prompt(past_guesses)
    completion = generate_stream(prompt, adapter_id)
    match = re.search(
        r"<guess>\s*(.*?)\s*</guess>", completion, re.DOTALL
    )
    if not match:
        raise RuntimeError("invalid guess")
    
    guess = match.group(1).upper()
    feedback = get_feedback(guess, secret_word)
    past_guesses.append(GuessWithFeedback(guess, feedback))
    print("\n\n")
    print(("-" * 100) + "\n")
    for past_guess in past_guesses:
        print(past_guess)
    
    if guess == secret_word:
        print("🎉 SUCCESS 🎉")
    elif len(past_guesses) >= 6:
        print("❌ better luck next time... ❌")

Try playing with the base model:

In [13]:
secret_word = "BRICK"
past_guesses = []
adapter_id = ""

In [14]:
next_turn(past_guesses, secret_word, adapter_id)

 
- This is the first guess, so we need to choose a word that covers a wide range of letters to maximize our chances of getting feedback.
- A good starting word could be "CRANE" because it contains a mix of vowels and consonants, and includes some common letters.
</think>
<guess> CRANE </guess>



----------------------------------------------------------------------------------------------------

CRANE → Feedback: C(-) R(✓) A(x) N(x) E(x)


Now try with the finetuned model:

In [15]:
secret_word = "BRICK"
past_guesses = []
adapter_id = "wordle-dlai/2"

In [ ]:
next_turn(past_guesses, secret_word, adapter_id)

I need to make a first guess for a 5-letter word in Wordle. For the first guess, it's best to choose a word that:



## Try for yourself!

Try different secret words above and see how the model responds. 